In [6]:
%load_ext autoreload
%autoreload 2

from meri import MERI, MERI_CONFIGS_PATH
import json
import os
import sys

from fmu_gen_hex.callbacks import *
from fmu_gen_hex.data_extraction import param_dict_to_param_formatted_param_list
from fmu_gen_hex.data_extraction import convert_tuple_units, convert_units, convert_side_units

base_path = os.path.abspath(os.path.join(os.pardir))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extract parameters from datasheet

In [7]:
pdf_path = os.path.join(base_path, "demo/data/he-specification.pdf")
schema_path = os.path.join(base_path, "demo/data/he-spec_schema.json")

with open(schema_path) as f:
    schema = json.load(f)

# use default configurartion
config_path=os.path.join(MERI_CONFIGS_PATH, "meri_yolo.yaml")

meri = MERI(pdf_path=pdf_path, config_yaml_path=config_path)

# populate provided json schema
extracted_parameters = meri.run(json.dumps(schema))

Running configuration:  <bound method AttrDict.to_dict of {'DEVICE': 'cuda',
 'LIB': 'PT',
 'PT': {'LAYOUT': {'FILTER': [], 'WEIGHTS': 'layout/docalynet/yolov10x_best.pt'}}}>


[1010 08:54.49 @doctectionpipe.py:84]  INF  Processing he-specification_0.pdf


Building pipeline from components:  [<meri.layout.pipeline_components.add_pdf_info_component.AddPDFInfoComponent object at 0x7fb13a440f80>, <meri.layout.pipeline_components.layout_detection_component.LayoutDetectorComponent object at 0x7fb287560830>]


[1010 08:54.50 @context.py:126]  INF  AddPDFInfoComponent total: 0.1529 sec.



0: 480x640 1 Caption, 1 Picture, 42.6ms
Speed: 10.6ms preprocess, 42.6ms inference, 41.6ms postprocess per image at shape (1, 3, 480, 640)


...

Actual Token Usage gpt-4o-mini: ('completion_tokens', 1103)
Round:  1


Processing content chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Estimated input token:  1932


[1010 08:55.26 @_client.py:1038]  INF  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing content chunks: 100%|██████████| 1/1 [00:08<00:00,  8.64s/it]

Actual Token Usage gpt-4o-mini: ('completion_tokens', 476)


## Format parameters

In [8]:
params = param_dict_to_param_formatted_param_list(extracted_parameters)

## Find best K

In [9]:
ax, K = optimize_callback(*params, 10, 10000)

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 13397.000000000004
             x: [ 1.000e+04]
           nit: 24
          nfev: 71
 final_simplex: (array([[ 1.000e+04],
                       [ 1.000e+04]]), array([ 1.340e+04,  1.340e+04]))
    Time  Temperature       Name
0    0.0          0.0  Cold Side
1    0.1          0.0  Cold Side
2    0.2          0.0  Cold Side
3    0.3          0.0  Cold Side
4    0.4          0.0  Cold Side
5    0.5          0.0  Cold Side
6    0.6          0.0  Cold Side
7    0.7          0.0  Cold Side
8    0.8          0.0  Cold Side
9    0.9          0.0  Cold Side
10   1.0         68.0  Cold Side
11   0.0         91.0   Hot Side
12   0.1          0.0   Hot Side
13   0.2          0.0   Hot Side
14   0.3          0.0   Hot Side
15   0.4          0.0   Hot Side
16   0.5          0.0   Hot Side
17   0.6          0.0   Hot Side
18   0.7          0.0   Hot Side
19   0.8          0.0   Hot Side
20

## Compile FMU and export

In [10]:
target_dir = os.path.join(base_path, "examples")
export_fmu_callback(100,10,"Parallel flow",4190,4190,68,91,46,56,400,400,target_dir)

-- Configuring done
-- Generating done
-- Build files have been written to: /workspaces/fmu-gen-hex/fmu_gen_hex/hex_delta55/build
Scanning dependencies of target hex_delta55
[ 25%] Building CXX object CMakeFiles/hex_delta55.dir/src/hex_delta55.cpp.o
[ 50%] Linking CXX shared library libhex_delta55.so
[100%] Built target hex_delta55


'/workspaces/fmu-gen-hex/examples/hex_delta55_exported.fmu'